In [3]:
#Authors: Josh Domeyer & Eda Zhang
#Date: 10/25/2018
#Project: NLTK 2-Week Project

#Description:
#
#
#

In [1]:
import re
import html5lib
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np
import nltk #nltk.download()
from nltk.corpus import stopwords
import wikipedia

In [280]:
#
# Scrapes list of wikipedia "list of disney theatrical animated features"
# and captures the corresponding summary of each of their pages
#

disney_movies = wikipedia.page("List of Disney theatrical animated features")
table_scrape = BeautifulSoup(disney_movies.html(), 'html.parser')
all_tables = table_scrape.findAll("table", "wikitable")
movies_table = pd.read_html(str(all_tables[1]), header = 0)

link_list = []
for link in all_tables[1].findAll('a'):
    link_list.append(link.get('href'))
    
link_list = [i.strip('/wiki/').
             replace('_', ' ').
             replace('%27', '\'').
             replace('%26', '&') for i in link_list if ('wiki' in i) & ('#' not in i) & 
             ('Productions' not in i) & ('Studios' not in i) & ('Animation' not in i) & 
             ('Motion Picture' not in i) & ('ImageMovers' not in i) & ('Entertainment' not in i) & 
             ('Films' not in i) & ('UTV' not in i) & ('C.O.R.E.' not in i)]

movies_df = pd.DataFrame(movies_table[0]).iloc[:,0:2]
movies_df['WikiLink'] = link_list
movies_df = movies_df.rename(index=str, columns={"Title": "Title", 
                                     "Original theatrical release date[rls 1]": "Release", 
                                     "WikiLink": "WikiLink"})

movies_df.iloc[6,2] = movies_df.iloc[6,2].replace('Bamb','Bambi')

movie_pages = [wikipedia.page(i) for i in movies_df.iloc[:,2]]
movies_df['Summary'] = [i.summary for i in movie_pages]

movies_df

,Title,Release,WikiLink,Summary
0,Academy Award Review of Walt Disney Cartoons,"May 19, 1937",Academy Award Review of Walt Disney Cartoons,Academy Award Review of Walt Disney Cartoons w...
1,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White and the Seven Dwarfs (1937 film),Snow White and the Seven Dwarfs is a 1937 Amer...
2,Pinocchio,"February 7, 1940",Pinocchio (1940 film),Pinocchio is a 1940 American animated musical ...
3,Fantasia,"November 13, 1940[rls 2]",Fantasia (1940 film),Fantasia is a 1940 American animated film prod...
4,The Reluctant Dragon,"June 20, 1941",The Reluctant Dragon (1941 film),The Reluctant Dragon is a 1941 American live a...
5,Dumbo,"October 23, 1941",Dumbo,Dumbo is a 1941 American animated film produce...
6,Bambi,"August 13, 1942",Bambi,Bambi is a 1942 American animated film directe...
7,Saludos Amigos,"August 24, 1942",Saludos Amigos,"Saludos Amigos (Spanish for ""Greetings, Friend..."
8,Victory Through Air Power,"July 17, 1943[5]",Victory Through Air Power (film),Victory Through Air Power is a 1943 American T...
9,The Three Caballeros,"December 21, 1944",The Three Caballeros,The Three Caballeros is a 1944 American live-a...


In [27]:
#Remove stopwords
#Lemma & stemming

In [ ]:
#
# Extra code from my first attempt at characters
#

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

disney_lines = [p for p in strip_tags(
    wikipedia.page("List of Disney characters").html()).split('\n') if p]

disney_lines_clean = []
for line in disney_lines:
    disney_lines_clean.append(re.sub(r'([^a-zA-Z ]+?)', '', line))

disney_chars = [re.sub(r'([^a-zA-Z ]+?)', '', i).strip().lower() 
                for i in nltk.sent_tokenize('. '.join(disney_lines_clean)) 
                if (len(i) > 4) & (len(i) < 15) & ('None' not in i) 
                & ('The' not in i) & ('Unknown' not in i)]

#disney_chars = list(set(disney_chars))
#disney_chars = [i for i in disney_chars if wikipedia.search(i, results = 1, suggestion = False)]

#disney_chars.sort()

#with open('./disney_chars.txt', 'w') as f:
#    for item in disney_chars:
#        f.write("%s\n" % item)

#disney_chars


#Examples of Wikipedia package operations

#print(wikipedia.summary("Adolf Hitler", sentences = 1))
#wikipedia.search("Adolf Hitler", results = 20)

#adolf = wikipedia.page("Adolf Hitler")
#adolf.title
#adolf.url
#adolf.images[0]
#adolf.content #Entire text
#adolf.links[10]

#print(wikipedia.suggest("Barack Obama"))